In [1]:
import numpy as np
import fmlib
import scipy.stats
from grading_tools import auto_marking_message

fmlib.test_all()

# Exercises

## Exercise

Price a digital call option by the method of writing an appropriate payoff function and using
the general function `price_by_monte_carlo` which you can find in fmlib. You should also write tests for all your functions.

In [2]:
def digital_call_payoff( S, K ):
    """S will be a matrix of stock price scenarios, rows representing scenarios, columns representing time"""
    ### BEGIN SOLUTION
    S_T = S[:,-1]
    return np.where(S_T>K,1.0,0.0)
    ### END SOLUTION

In [3]:
def test_digital_call_payoff():
    ### BEGIN SOLUTION
    S = np.array([[1,1,1],[2,3,4]])
    actual = digital_call_payoff(S, 2)
    expected = np.array([0.0,1.0])
    np.testing.assert_almost_equal(actual, expected )
    ### END SOLUTION

In [4]:
def price_digital_call_by_monte_carlo( S0, r, sigma, K, T, n_steps, n_paths ):
    """This function should return lower and upper to give a 99% confidence interval for the price"""
    ### BEGIN SOLUTION
    def payoff( S ):
        return digital_call_payoff( S, K )
    return fmlib.price_by_monte_carlo(S0, r, sigma, T, n_steps, n_paths, payoff )
    ### END SOLUTION

In [5]:
def test_price_digital_call_by_monte_carlo():
    ### BEGIN SOLUTION
    np.random.seed(0)
    K = 105; T = 1;
    S0 = 100; r = 0.05; sigma = 0.25
    n_steps = 365; n_paths = 100000;
    low,high = price_digital_call_by_monte_carlo(S0, r,sigma,K, T,1, n_paths)
    d1,d2 = fmlib.compute_d1_and_d2(S0,0,K,T,r,sigma)
    bs_price = np.exp(-r*T)*fmlib.N( d2 )
    assert low<bs_price
    assert bs_price<high
    ### END SOLUTION

In [6]:
test_digital_call_payoff()
auto_marking_message()

Auto marking message: 🌟 Correct


In [7]:
test_price_digital_call_by_monte_carlo()
auto_marking_message()

Auto marking message: 😻 Correct


In [8]:
### BEGIN HIDDEN TESTS
test_price_digital_call_by_monte_carlo()
error = False
oldF = price_digital_call_by_monte_carlo
try:
    def price_digital_call_by_monte_carlo( S0, r, sigma, K, T, n_steps, n_paths):
        return 2,3
    test_price_digital_call_by_monte_carlo()
except:
    error = True
finally:
    price_digital_call_by_monte_carlo = oldF
if not error:
    raise Exception("Test of pricing digital call isn't working")
### END HIDDEN TESTS

In [9]:
### BEGIN HIDDEN TESTS
test_digital_call_payoff()
error = False
oldF = digital_call_payoff
try:
    def digital_call_payoff( S, K ):
        return  S[:,-1]*0
    test_digital_call_payoff()
except:
    error = True
finally:
    digital_call_payoff = oldF
if not error:
    raise Exception("Test of digital call payoff isn't working")
### END HIDDEN TESTS

In [10]:
### BEGIN HIDDEN TESTS
def my_test_digital_call_payoff():
    S = np.array([[1,1,1],[2,3,4]])
    actual = digital_call_payoff(S, 2)
    expected = np.array([0.0,1.0])
    np.testing.assert_almost_equal(actual, expected )
my_test_digital_call_payoff()
### END HIDDEN TESTS

In [11]:
### BEGIN HIDDEN TESTS
def my_test_price_digital_call_by_monte_carlo():
    np.random.seed(0)
    K = 105; T = 1;
    S0 = 100; r = 0.05; sigma = 0.25
    n_steps = 365; n_paths = 100000;
    low,high = price_digital_call_by_monte_carlo(S0, r,sigma,K, T,1, n_paths)
    d1,d2 = fmlib.compute_d1_and_d2(S0,0,K,T,r,sigma)
    bs_price = np.exp(-r*T)*fmlib.N( d2 )
    assert low<bs_price
    assert bs_price<high
    
my_test_price_digital_call_by_monte_carlo()
### END HIDDEN TESTS

## Exercise

Compute an estimate for the price of an up and out knockout option with strike $K=95$, barrier $B=130$ and maturity $T=1$
in the Black-Scholes model with parameters $S_0=100$, $r=0.05$, $\sigma=0.25$ using the Monte Carlo method with
365 steps and 100000 simulations. Store your estimate in a variable called `price`.

Test your code througoughly.

In [12]:
S0 = 100
r = 0.05
sigma=0.25
B = 130
K = 95
T = 1
n_steps = 365
n_paths = 100000

### BEGIN SOLUTION
def knockout_payoff( S, K, B ):
    not_knocked_out = np.max(S, axis=1)<B
    S_T = S[:,-1]
    call_payoff = np.maximum( S_T-K, 0 )
    payoff = not_knocked_out*call_payoff
    return payoff

def test_knockout_payoff():
    # Test the payoff function in isolation
    S = np.array([[100,105,110],
                  [100,120,110],
                  [100,105,100]])
    K = 105
    B = 115
    actual = knockout_payoff(S, K, B)
    expected = np.array([5,0,0])
    np.testing.assert_almost_equal(actual,expected)
    
test_knockout_payoff()

def price_knockout_by_monte_carlo( S0, r, sigma, K, T, B, n_steps, n_paths ):
    """This function should return lower and upper to give a 99% confidence interval for the price"""
    def payoff( S ):
        return knockout_payoff( S, K, B )
    return fmlib.price_by_monte_carlo(S0, r, sigma, T, n_steps, n_paths, payoff )

def test_price_knockout_by_monte_carlo():
    # Choose a high barrier, then the knock out becomes a call option so
    # we can test we get the correct price in this case
    np.random.seed(0)
    S0 = 100
    r = 0.05
    sigma=0.25
    B = 100000
    K = 105
    T = 1
    n_steps = 365
    n_paths = 10000
    lower, upper = price_knockout_by_monte_carlo(S0,r,sigma,K,T,B,n_steps,n_paths)
    price = fmlib.black_scholes_call_price(S0,0,K,T,r,sigma)
    assert lower<price
    assert upper>price
    
    # If the barrier=K, the option is worthless
    lower, upper = price_knockout_by_monte_carlo(S0,r,sigma,K,T,K,n_steps,n_paths)
    assert upper<0.00000001

test_price_knockout_by_monte_carlo()
lower, upper = price_knockout_by_monte_carlo(S0,r,sigma,K,T,B,n_steps,n_paths)
price = 0.5*(lower+upper)
### END SOLUTION

In [13]:
assert price>0
# BEGIN HIDDEN TESTS
assert price > 3.6
assert price < 3.9
# END HIDDEN TESTS
auto_marking_message()

Auto marking message: 😊 Correct


## Exercise

It is important to be able to calculate the Greeks too. A trader will need to know the delta of a derivative in order to carry out the replication strategy.

Suppose that $X$ and $Y$ are two random variables and you wish to estimate $E(X-Y)$. The *wrong* way to do this is to first estimate $E(X)$ by a simulation of $X$, then estimate $E(Y)$ using a simulation of $Y$, then compute the difference. The reason is that each estimate is just an estimate and so will contain some random error. If $|E(X)-E(Y)|$ is less than this random error, then this will dominate the calculation. Even if the difference is large, you will be unnecessarily increasing the amount of random error. The correct approach is to simulate $X-Y$ directly.

An equivalent approach to simulating $X-Y$ directly is to simulate $X$ and $Y$ separately but using the same scenarios in each simulation. One can then compute $E(X-Y)$ by taking the difference of the averages of $X$ and $Y$.

To compute the Delta of an option by Monte Carlo you should choose a reasonably small value of $h$ (say $h=S_0 \times 10^{-5}$) and compute the price of the option when the first simulated stock price satisfies $\tilde{S}_0=S_0-h$ and also when $\tilde{S}_0=S_0+h$ but
using the same scenarios for $W^{\mathbb Q}_t$ in both simulations. You can then estimate the Delta using the central difference estimate for the derivative.

Use this approach to compute the delta of a call option by Monte Carlo and test that your answer is correct.

In [14]:
def delta_by_monte_carlo( S0, r, sigma, T, n_steps, n_paths, payoff_function ):
    h = S0*10**(-5)
    S_twiddle, times = fmlib.simulate_gbm_paths(S0, r, sigma, T, n_steps, n_paths )
    paths_for_S_minus_h = S_twiddle/S0*(S0-h)
    paths_for_S_plus_h = S_twiddle/S0*(S0+h)
    payoffs_S_plus_h = np.exp(-r*T)*payoff_function(paths_for_S_plus_h)
    payoffs_S_minus_h = np.exp(-r*T)*payoff_function(paths_for_S_minus_h)
    samples = (payoffs_S_plus_h-payoffs_S_minus_h)/(2*h)
    p = 99
    alpha = scipy.stats.norm.ppf((1-p/100)/2)
    delta = np.mean( samples )
    sigma_sample = np.std( samples )
    lower = delta + alpha*sigma_sample/np.sqrt(n_paths)
    upper = delta - alpha*sigma_sample/np.sqrt(n_paths)
    return lower, upper

def delta_call_confidence_interval_by_monte_carlo( S0, r, sigma, K, T, n_paths ):
    def call_payoff( S ):
        S_T = S[:,-1]
        return np.maximum( S_T-K, 0 )
    return delta_by_monte_carlo(S0, r, sigma, T, 1, n_paths, call_payoff )

In [15]:
def delta_call_by_monte_carlo( S0, r, sigma, K, T, n_paths ):
    ### BEGIN SOLUTION
    low, high = delta_call_confidence_interval_by_monte_carlo(S0, r, sigma, K, T, n_paths )
    return 0.5*(low+high)
    ### END SOLUTION

def test_delta_call_by_monte_carlo():
    ### BEGIN SOLUTION
    np.random.seed(0)
    K = 105; T = 1;
    S0 = 100; r = 0.05; sigma = 0.25
    n_paths = 100000;
    low,high = delta_call_confidence_interval_by_monte_carlo(S0, r,sigma,K, T, n_paths)
    bs_delta = fmlib.black_scholes_delta(S0,0,K,T,r,sigma)
    assert low<bs_delta
    assert bs_delta<high
    ### END SOLUTION


In [16]:
test_delta_call_by_monte_carlo()
### BEGIN HIDDEN TESTS
K = 105; T = 1;
S0 = 100; r = 0.05; sigma = 0.25
n_paths = 100000;
d1 = delta_call_by_monte_carlo(S0,r,sigma,K,T,n_paths)
d2 = delta_call_by_monte_carlo(S0,r,sigma,K,T,n_paths)
assert d1>0.53
assert d1<0.57
assert abs(d1-d2)>0.0000001 # Deterministic answers aren't OK
### END HIDDEN TESTS
auto_marking_message()

Auto marking message: 😍 Correct
